<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_2/03_simple_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルなBERTの実装
訓練済みのモデルを使用し、文章の一部の予測、及び2つの文章が連続しているかどうかの判定を行います。

## ライブラリのインストール
PyTorch-Transformers、および必要なライブラリのインストールを行います。

In [ ]:
!pip install folium==0.2.1
!pip install urllib3==1.25.11
!pip install pytorch-transformers==1.2.0

## 文章の一部の予測
文章における一部の単語をMASKし、それをBERTのモデルを使って予測します。

In [ ]:
import torch
from pytorch_transformers import BertForMaskedLM
from pytorch_transformers import BertTokenizer


text = "[CLS] I played baseball with my friends at school yesterday [SEP]"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
words = tokenizer.tokenize(text)
print(words)

文章の一部をMASKします。

In [ ]:
msk_idx = 3
words[msk_idx] = "[MASK]"  # 単語を[MASK]に置き換える
print(words)

単語を対応するインデックスに変換します。

In [ ]:
word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
word_tensor = torch.tensor([word_ids])  # テンソルに変換
print(word_tensor)

BERTのモデルを使って予測を行います。

In [ ]:
msk_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
msk_model.cuda()  # GPU対応
msk_model.eval()

x = word_tensor.cuda()  # GPU対応
y = msk_model(x)  # 予測
result = y[0]
print(result.size())  # 結果の形状

_, max_ids = torch.topk(result[0][msk_idx], k=5)  # 最も大きい5つの値
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())  # インデックスを単語に変換
print(result_words)

## 文章が連続しているかどうかの判定
BERTのモデルを使って、2つの文章が連続しているかどうかの判定を行います。  
以下の関数`show_continuity`では、2つの文章の連続性を判定し、表示します。

In [ ]:
from pytorch_transformers import BertForNextSentencePrediction

def show_continuity(text, seg_ids):
    words = tokenizer.tokenize(text)
    word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
    word_tensor = torch.tensor([word_ids])  # テンソルに変換

    seg_tensor = torch.tensor([seg_ids])

    nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
    nsp_model.cuda()  # GPU対応
    nsp_model.eval()

    x = word_tensor.cuda()  # GPU対応
    s = seg_tensor.cuda()  # GPU対応

    y = nsp_model(x, s)  # 予測
    result = torch.softmax(y[0], dim=1)
    print(result)  # Softmaxで確率に
    print(str(result[0][0].item()*100) + "%の確率で連続しています。")

`show_continuity`関数に、自然につながる2つの文章を与えます。

In [ ]:
text = "[CLS] What is baseball ? [SEP] It is a game of hitting the ball with the bat [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

`show_continuity`関数に、自然につながらない2つの文章を与えます。

In [ ]:
text = "[CLS] What is baseball ? [SEP] This food is made with flour and milk [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)